# Classifying OUV using GRU sequence model + Attention

## Imports

In [1]:
import sys
sys.executable

'/opt/miniconda3/bin/python'

In [2]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

from torch.autograd import Variable

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

from scipy.special import softmax

import pickle
import matplotlib.pyplot as plt

import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer('spacy')

In [3]:
print("PyTorch version {}".format(torch.__version__))
print("GPU-enabled installation? {}".format(torch.cuda.is_available()))

PyTorch version 1.7.0
GPU-enabled installation? False


In [4]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


## Data Vectorization Classes

### The Vocabulary

In [5]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
                
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [6]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

### The Vectorizer

In [7]:
class OuvVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, ouv_vocab):
        """
        Args:
            review_vocab (Vocabulary): maps words to integers
        """
        self.ouv_vocab = ouv_vocab
        
    def vectorize(self, data, vector_length = -1):
        """Create a collapsed one-hit vector for the ouv data
        
        Args:
            data (str): the ouv description data
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vectorized data (np.ndarray)
        """
        indices = []
        indices.extend(self.ouv_vocab.lookup_token(token) for token in data.split(' '))
        #indices.append(self.ouv_vocab.end_seq_index)
        
        if vector_length < 0:
            vector_length = len(indices)
            
        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.ouv_vocab.mask_index
        
        return out_vector, len(indices)

    @classmethod
    def from_dataframe(cls, ouv_df, cutoff=5):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            ouv_df (pandas.DataFrame): the ouv dataset
            cutoff (int): the parameter for frequency-based filtering
        Returns:
            an instance of the OuvVectorizer
        """
        
        # Add top words if count > provided count
        word_counts = Counter()
        for data in ouv_df.data:
            for word in data.split(' '):
                if word not in string.punctuation:
                    word_counts[word] += 1
        
        ouv_vocab = SequenceVocabulary()
        for word, count in word_counts.items():
            if count > cutoff:
                ouv_vocab.add_token(word)

        return cls(ouv_vocab)

    @classmethod
    def from_serializable(cls, contents):
        """Instantiate a OuvVectorizer from a serializable dictionary
        
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the OuvVectorizer class
        """
        ouv_vocab = SequenceVocabulary.from_serializable(contents['ouv_vocab'])
        
        return cls(ouv_vocab=ouv_vocab)

    def to_serializable(self):
        """Create the serializable dictionary for caching
        
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'ouv_vocab': self.ouv_vocab.to_serializable()}

### The Dataset

In [8]:
class OuvDataset(Dataset):
    def __init__(self, ouv_df, vectorizer):
        """
        Args:
            ouv_df (pandas.DataFrame): the dataset
            vectorizer (ReviewVectorizer): vectorizer instantiated from dataset
        """
        self.ouv_df = ouv_df
        self._vectorizer = vectorizer
        
        # +0 if not using begin_seq and end seq, +1 if only using begin_seq, +2 if using both begin and end seq tokens
        measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, ouv_df.data)) + 0

        self.train_df = self.ouv_df[self.ouv_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.ouv_df[self.ouv_df.split=='dev']
        self.validation_size = len(self.val_df)

        self.test_df = self.ouv_df[self.ouv_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, ouv_csv, cutoff):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            ouv_csv (str): location of the dataset
            cutoff (int): the boundary to set the words into unknown
        Returns:
            an instance of OuvDataset
        """
        ouv_df = pd.read_csv(ouv_csv)
        train_ouv_df = ouv_df[ouv_df.split=='train']
        return cls(ouv_df, OuvVectorizer.from_dataframe(train_ouv_df, cutoff=cutoff))
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, ouv_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            ouv_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of OuvDataset
        """
        ouv_df = pd.read_csv(ouv_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(ouv_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of ReviewVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return OuvVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe 
        
        Args:
            split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and component for labels (y_target and y_fuzzy)
        """
        row = self._target_df.iloc[index]

        ouv_vector, vec_length = \
            self._vectorizer.vectorize(row.data, self._max_seq_length)

        true_label = \
            np.fromstring(row.true[1:-1],dtype=float, sep=' ')
        
        if len(true_label)==10:
            true_label = np.append(true_label,0.0)
        
        fuzzy_label = \
            np.fromstring(row.fuzzy[1:-1],dtype=float, sep=' ')

        return {'x_data': ouv_vector,
                'y_target': true_label,
                'y_fuzzy': fuzzy_label,
                'x_length': vec_length
               }

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size  
    
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## The Model: GRU sequential Model

In [9]:
def batch_matmul_bias(seq, weight, bias, nonlinearity=''):
    s = None
    bias_dim = bias.size()
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight) 
        _s_bias = _s + bias.expand(bias_dim[0], _s.size()[0]).transpose(0,1)
        if(nonlinearity=='tanh'):
            _s_bias = torch.tanh(_s_bias)
        _s_bias = _s_bias.unsqueeze(0)
        if(s is None):
            s = _s_bias
        else:
            s = torch.cat((s,_s_bias),0)
    return s.squeeze()

In [10]:
def batch_matmul(seq, weight, nonlinearity=''):
    s = None
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight)
        if(nonlinearity=='tanh'):
            _s = torch.tanh(_s)
        _s = _s.unsqueeze(0)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)
    return s.squeeze()

In [11]:
def attention_mul(rnn_outputs, att_weights):
    attn_vectors = None
    for i in range(rnn_outputs.size(0)):
        h_i = rnn_outputs[i]
        a_i = att_weights[i].unsqueeze(1).expand_as(h_i)
        h_i = a_i * h_i
        h_i = h_i.unsqueeze(0)
        if(attn_vectors is None):
            attn_vectors = h_i
        else:
            attn_vectors = torch.cat((attn_vectors,h_i),0)
    return torch.sum(attn_vectors, 0).unsqueeze(0)

In [12]:
class AttnGRUClassifier(nn.Module):
    
    def __init__(self, batch_size, embedding_size, num_embeddings, hidden_dim, num_classes, dropout_p,
                 batch_first=True, pretrained_embeddings=None, padding_idx=0, bi = False, freeze=True):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(AttnGRUClassifier, self).__init__()

        if pretrained_embeddings is None:

            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding.from_pretrained(pretrained_embeddings,
                                    padding_idx=padding_idx,
                                    freeze=True)
        
        self.batch_size = batch_size
        self.batch_first = batch_first
        self.dropout = nn.Dropout(dropout_p)
        self.padding_idx = padding_idx
        self.hidden_dim = hidden_dim
        self.bi = bi
        
        self.word_gru = nn.GRU(input_size = embedding_size, hidden_size=hidden_dim, batch_first=False,
                         num_layers=1, bidirectional = bi)
        
        self.word_bias = nn.Linear((bi+1)*hidden_dim, (bi+1)*hidden_dim)
        self.word_n_bias = nn.Linear((bi+1)*hidden_dim, 1, bias=False)
        
        self.fc = nn.Linear(hidden_dim * (bi+1), num_classes)

    def forward(self, x_in, state_word, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        
        # embed and permute so features are channels
        x_embedded = self.emb(x_in)
        padding_mask = x_in.eq(self.padding_idx)
        length = (~padding_mask).sum(dim=1)
        
        if self.batch_first:
            x_embedded = x_embedded.transpose(1,0)
        
        # rnn units
        output_word, state_word = self.word_gru(x_embedded, state_word)
        #(T,B,C)
        word_squish = self.word_bias(output_word.view(-1,output_word.shape[-1]))
        #(B*T,C)
        word_squish = torch.tanh(word_squish)
        word_attn = self.word_n_bias(word_squish).view(output_word.shape[0],output_word.shape[1],1)
        word_attn[padding_mask.transpose(0,1)]= -np.inf
        #(B*T,1)->(T,B,1)
        
        word_attn_norm = F.softmax(word_attn, dim=0).transpose(0,1)
        #(B,T,1)
        
        word_attn_vec = torch.bmm(output_word.permute(1,2,0),word_attn_norm)
        #(B,C,T)*(B,T,1)->(B,C,1)
        
        y_out = self.fc(self.dropout(word_attn_vec.squeeze()))
        
        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)
            
        return y_out, state_word, word_attn_norm.squeeze()
    
    def init_hidden(self):
        return Variable(torch.zeros((self.bi+1), self.batch_size, self.hidden_dim))

### Helper Functions

In [13]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_k_acc_val': 0,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_1_acc': [],
            'train_k_acc': [],
            'train_k_jac': [],
            'val_loss': [],
            'val_1_acc': [],
            'val_k_acc': [],
            'val_k_jac': [],
            'test_loss': -1,
            'test_1_acc': -1,
            'test_k_acc':-1,
            'test_k_jac':-1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        acc_tm1, acc_t = train_state['val_k_acc'][-2:]

        # If accuracy worsened
        if acc_t <= train_state['early_stopping_best_k_acc_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model from sklearn
            if acc_t > train_state['early_stopping_best_k_acc_val']:
                train_state['early_stopping_best_k_acc_val'] = acc_t
                torch.save(model.state_dict(), train_state['model_filename'])
                
            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

### Evaluation Metrics

In [14]:
def compute_cross_entropy(y_pred, y_target):
    y_target = y_target.cpu().float()
    y_pred = y_pred.cpu().float()
    criterion = nn.BCEWithLogitsLoss()
    return criterion(y_target, y_pred)

def compute_1_accuracy(y_pred, y_target):
    y_target_indices = y_target.max(dim=1)[1]
    y_pred_indices = y_pred.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target_indices).sum().item()
    return n_correct / len(y_pred_indices) * 100

def compute_k_accuracy(y_pred, y_target, k=3):
    y_pred_indices = y_pred.topk(k, dim=1)[1]
    y_target_indices = y_target.max(dim=1)[1]
    n_correct = torch.tensor([y_pred_indices[i] in y_target_indices[i] for i in range(len(y_pred))]).sum().item()
    return n_correct / len(y_pred_indices) * 100

def compute_k_jaccard_index(y_pred, y_target, k=3):
    y_target_indices = y_target.topk(k, dim=1)[1]
    y_pred_indices = y_pred.max(dim=1)[1]
    jaccard = torch.tensor([len(np.intersect1d(y_target_indices[i], y_pred_indices[i]))/
                            len(np.union1d(y_target_indices[i], y_pred_indices[i]))
                            for i in range(len(y_pred))]).sum().item()
    return jaccard / len(y_pred_indices)

def compute_jaccard_index(y_pred, y_target, k=3, multilabel=False):
    
    threshold = 1.0/(k+1)
    threshold_2 = 0.5
    
    if multilabel:
        y_pred_indices = y_pred.gt(threshold_2)
    else:
        y_pred_indices = y_pred.gt(threshold)
    
    y_target_indices = y_target.gt(threshold)
        
    jaccard = ((y_target_indices*y_pred_indices).sum(axis=1)/((y_target_indices+y_pred_indices).sum(axis=1)+1e-8)).sum().item()
    return jaccard / len(y_pred_indices)

In [15]:
def softmax_sensitive(T):
    T = np.exp(T) - np.exp(0) + 1e-9
    if len(T.shape)==1:
        return T/T.sum()
    return  T/(T.sum(axis=1).unsqueeze(1))

In [16]:
def cross_entropy(pred, soft_targets):
    logsoftmax = nn.LogSoftmax(dim=1)
    return torch.mean(torch.sum(- soft_targets * logsoftmax(pred), 1))

In [17]:
# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = args.device
    return torch.from_numpy(df.values).float().to(device)

In [18]:
def get_prior():
    prior = pd.read_csv(args.prior_csv,sep=';',names=classes[:-1], skiprows=1)
    prior['Others'] = 1
    prior = prior.T
    prior['Others'] = 1
    prior = df_to_tensor(prior)
    return(prior)

In [19]:
def compute_fuzzy_label(y_target, y_fuzzy, fuzzy=False, how='uni', lbd=0):
    '''
    Using two sets of prediction labels and fuzziness parameters to compute the fuzzy label in the form as 
    a distribution over classes
    
    Args:
    y_target (torch.Tensor) of shape (n_batch, n_classes): the true label of the ouv description
    y_fuzzy (torch.Tensor) of shape (n_batch, n_classes): the fuzzy label of the ouv description
    fuzzy (bool): whether or not to turn on the fuzziness option
    how (string): the way fuzziness weights are used, one of the options in {'uni', 'prior'}
    lbd (float): the scaler applied to the fuzziness of the label
    
    Returns:
    A pytorch Tensor of shape (n_batch, n_classes): The processed label in the form of distribution that add to 1
    '''
    assert y_target.shape == y_fuzzy.shape, 'target labels must have the same size'
    assert how in {'uni', 'prior', 'origin'}, '''how must be one of the two options in {'uni', 'prior'}'''
    
    if not fuzzy:
        return softmax_sensitive(y_target)
    
    if how == 'uni':
        y_label = y_target + lbd * y_fuzzy
        return softmax_sensitive(y_label)
    
    ### TO DO ###
    elif how == 'prior':
        prior = get_prior()
        y_inter = torch.matmul(y_target.float(),prior)
        y_inter = y_inter/(y_inter.max(dim=1, keepdim=True)[0])
        y_label = y_target + lbd * y_fuzzy * y_inter
        return softmax_sensitive(y_label)
    
    else:
        y_label = y_target + lbd
        return softmax_sensitive(y_label)

### General Utilities

In [20]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings 
    
    Args:
        glove_filepath (str): path to the glove embeddings file 
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    with open(glove_filepath, "r") as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0] 
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

### Settings and Some Prep Work

In [21]:
args = Namespace(
    # Data and Path information
    frequency_cutoff=1,
    model_state_file='model.pth',
    ouv_csv='Data/ouv_with_splits_full.csv',
    #ouv_csv='Data/all_with_splits_full.csv',
    prior_csv = 'Data/Coappearance_matrix.csv',
    ratings_csv = 'Data/human_rates.csv',
    save_dir='model_storage/attn/',
    vectorizer_file='vectorizer.json',
    # Model hyper parameters
    glove_filepath='Data/glove/glove.6B.300d.txt', 
    use_glove=True,
    freeze = True,
    embedding_size=300, 
    hidden_dim=128, 
    bi = False,
    # Training hyper parameters
    batch_size=256,
    early_stopping_criteria=5,
    learning_rate=0.001,
    l2=1e-5,
    dropout_p=0,
    k = 3,
    fuzzy = True,
    fuzzy_how = 'uni',
    fuzzy_lambda = 0.2,
    num_epochs=100,
    seed=1337,
    # Runtime options
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

classes = ['Criteria i', 'Criteria ii', 'Criteria iii', 'Criteria iv', 'Criteria v', 'Criteria vi', 
              'Criteria vii', 'Criteria viii', 'Criteria ix', 'Criteria x', 'Others']

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

print("Using CUDA: {}".format(args.cuda))

args.device = torch.device("cuda" if args.cuda else "cpu")

# Set seed for reproducibility
#set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

Expanded filepaths: 
	model_storage/attn/vectorizer.json
	model_storage/attn/model.pth
Using CUDA: False


## Initialization

In [23]:
if args.reload_from_files:
    # training from a checkpoint
    dataset = OuvDataset.load_dataset_and_load_vectorizer(args.ouv_csv, args.vectorizer_file)

else:
    # create dataset and vectorizer
    dataset = OuvDataset.load_dataset_and_make_vectorizer(args.ouv_csv, cutoff=args.frequency_cutoff)
    dataset.save_vectorizer(args.vectorizer_file)    

vectorizer = dataset.get_vectorizer()
set_seed_everywhere(args.seed, args.cuda)
# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.ouv_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath, 
                                       words=words)
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None
classifier = AttnGRUClassifier(batch_size=args.batch_size,
                               embedding_size=args.embedding_size,
                               num_embeddings=len(vectorizer.ouv_vocab),
                            hidden_dim=args.hidden_dim, 
                            num_classes=len(classes), 
                            dropout_p=args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0,
                            bi = args.bi,
                            freeze = args.freeze)

Using pre-trained embeddings


## Loading Trained Model

### Option 1 LS Model

In [24]:
with open(args.save_dir+'hyperdict_fuzzy.p', 'rb') as fp:
    hyperdict_fuzzy = pickle.load(fp)
    train_state = hyperdict_fuzzy[('uni',0.2)]

In [25]:
classifier.load_state_dict(torch.load(args.save_dir+'1337/model.pth',map_location=torch.device('cpu')))
classifier.eval()

AttnGRUClassifier(
  (emb): Embedding(6047, 300, padding_idx=0)
  (dropout): Dropout(p=0, inplace=False)
  (word_gru): GRU(300, 128)
  (word_bias): Linear(in_features=128, out_features=128, bias=True)
  (word_n_bias): Linear(in_features=128, out_features=1, bias=False)
  (fc): Linear(in_features=128, out_features=11, bias=True)
)

### Option 2 Baseline w/o LS

In [42]:
with open(args.save_dir+'hyperdict_fuzzy.p', 'rb') as fp:
    hyperdict_fuzzy = pickle.load(fp)
    train_state = hyperdict_fuzzy[('uni',0)]

In [172]:
classifier.load_state_dict(torch.load(args.save_dir+'baseline/model.pth',map_location=torch.device('cpu')))
classifier.eval()

AttnGRUClassifier(
  (emb): Embedding(6047, 300, padding_idx=0)
  (dropout): Dropout(p=0, inplace=False)
  (word_gru): GRU(300, 128)
  (word_bias): Linear(in_features=128, out_features=128, bias=True)
  (word_n_bias): Linear(in_features=128, out_features=1, bias=False)
  (fc): Linear(in_features=128, out_features=11, bias=True)
)

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(classifier)

183179

In [45]:
set_seed_everywhere(args.seed, args.cuda)
# compute the loss & accuracy on the test set using the best available model
loss_func = cross_entropy
#train_state = hyperdict[best_config]
#classifier.load_state_dict(torch.load(train_state['model_filename']))
#classifier = classifier.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_1_acc = 0.
running_k_acc = 0.
running_k_jac = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    state_word = classifier.init_hidden().to(args.device)
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # step 3. compute the output
    y_pred,state_word,_ = classifier(X, state_word)


    # compute the loss
    loss = loss_func(y_pred, Y)
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_1_t = compute_1_accuracy(y_pred, Y)
    acc_k_t = compute_k_accuracy(y_pred, Y, args.k)
    jac_k_t = compute_jaccard_index(y_pred, Y, args.k)

    running_1_acc += (acc_1_t - running_1_acc) / (batch_index + 1)
    running_k_acc += (acc_k_t - running_k_acc) / (batch_index + 1)
    running_k_jac += (jac_k_t - running_k_jac) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_1_acc'] = running_1_acc
train_state['test_k_acc'] = running_k_acc
train_state['test_k_jac'] = running_k_jac

In [41]:
#LS Model
train_state

{'stop_early': True,
 'early_stopping_step': 5,
 'early_stopping_best_k_acc_val': 91.796875,
 'learning_rate': 0.001,
 'epoch_index': 15,
 'train_loss': [2.198943771596139,
  1.7894909588827341,
  1.6077453631286176,
  1.51393375583656,
  1.4374524229319623,
  1.3765786128704782,
  1.3162432918650375,
  1.2749618684771529,
  1.2391568531602377,
  1.2141996012523106,
  1.1801471967092538,
  1.164778186948496,
  1.1533445934247148,
  1.1398225917144127,
  1.1235810948209222,
  1.1185056999615688],
 'train_1_acc': [23.322610294117645,
  35.52389705882353,
  43.635110294117645,
  48.25367647058823,
  53.216911764705884,
  58.18014705882353,
  61.465992647058826,
  64.15441176470588,
  65.6939338235294,
  67.6470588235294,
  68.63511029411765,
  70.1516544117647,
  70.31250000000001,
  71.30055147058822,
  72.74816176470588,
  72.47242647058823],
 'train_k_acc': [54.940257352941174,
  71.92095588235296,
  80.67555147058825,
  83.61672794117648,
  87.61488970588235,
  89.65992647058822,
  91

In [46]:
#Baseline
train_state

{'stop_early': True,
 'early_stopping_step': 5,
 'early_stopping_best_k_acc_val': 91.6015625,
 'learning_rate': 0.001,
 'epoch_index': 15,
 'train_loss': [2.1776003070893335,
  1.7192871886354384,
  1.478551664518919,
  1.3462597753399255,
  1.232127564134482,
  1.1414252035724703,
  1.0566094870754879,
  0.9943482124976606,
  0.9379362961109343,
  0.8935166259121269,
  0.8489513760790851,
  0.8208194994694975,
  0.7761225418543545,
  0.7242527879920942,
  0.6912963192212869,
  0.6689500920684159],
 'train_1_acc': [23.253676470588236,
  35.66176470588236,
  43.79595588235294,
  49.333639705882355,
  54.18198529411764,
  58.478860294117645,
  61.4889705882353,
  64.49908088235294,
  66.75091911764706,
  67.578125,
  69.04871323529412,
  70.22058823529413,
  72.44944852941178,
  73.87408088235296,
  75.34466911764706,
  76.63143382352942],
 'train_k_acc': [54.526654411764696,
  71.36948529411765,
  80.53768382352939,
  83.73161764705883,
  87.68382352941177,
  89.45312499999999,
  90.602

## Inference

In [27]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [28]:
def predict_rating(text, classifier, vectorizer, classes, max_length=dataset._max_seq_length, k=1):
    """Predict the rating of a review
    
    Args:
        text (str): the text of the description
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        classes (list of str): The name of the ouv classes
        k (int): show the largest k prediction, default to 1
    """
    ouv = preprocess_text(text)
    
    classifier.eval()
    state_word = classifier.init_hidden().to(args.device)
    vectorized_ouv = torch.tensor(vectorizer.vectorize(ouv, vector_length = max_length)[0])
    X = vectorized_ouv.view(1,-1).repeat(args.batch_size,1,1)
    result = classifier(vectorized_ouv.unsqueeze(0).repeat(args.batch_size,1),state_word, apply_softmax=True)[0][0]
    
    if k==1:
        pred_id = result.argmax().item()
        return (classes[pred_id], result[pred_id])
    else:
        pred_indices = [i.item() for i in result.topk(k)[1]]
        output = []
        for pred_id in pred_indices:
            output.append((classes[pred_id],result[pred_id].item()))
        return output

In [29]:
test_ouv = 'this is a very old building dating back to 13th century'

prediction = predict_rating(test_ouv,classifier,vectorizer,classes)
print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

this is a very old building dating back to 13th century -> Criteria iv with a probability of 0.46


In [30]:
test_ouv = 'this is a very old building dating back to 13th century'
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k=k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
this is a very old building dating back to 13th century -> Criteria iv with a probability of 0.46
this is a very old building dating back to 13th century -> Criteria iii with a probability of 0.41
this is a very old building dating back to 13th century -> Criteria ii with a probability of 0.04


In [31]:
test_ouv = 'The particular layout of the complex is unique to this site'
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k=k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
The particular layout of the complex is unique to this site -> Criteria iv with a probability of 0.59
The particular layout of the complex is unique to this site -> Criteria ii with a probability of 0.10
The particular layout of the complex is unique to this site -> Criteria iii with a probability of 0.10


In [33]:
test_ouv = '''the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torcellos cathedral to the church of santa maria della salute . the years of the republics extraordinary golden 
age are represented by monuments of incomparable beauty san marco , palazzo ducale , san zanipolo , scuola di san 
marco , frari and scuola di san rocco , san giorgio maggiore , etc . '''
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k=k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torcellos cathedral to the church of santa maria della salute . the years of the republics extraordinary golden 
age are represented by monuments of incomparable beauty san marco , palazzo ducale , san zanipolo , scuola di san 
marco , frari and scuola di san rocco , san giorgio maggiore , etc .  -> Criteria i with a probability of 0.73
the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torcellos cathedral to the church of santa maria della salute . the years of the republics extraordinary golden 
age are represented by monuments of incomparable beauty san marco , palazzo ducale , san zanipolo , scuola di san 
marco , frari and scuola di san rocco , san giorgio maggiore , etc .  -> Criteria vii with a probability of 0.06
the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torc

In [55]:
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print('[%s]' % self.name,)
        print('Elapsed: %s' % (time.time() - self.tstart))
        
set_seed_everywhere(args.seed, args.cuda)        
test_ouv = 'The particular layout of the complex is unique to this site'
k=3
with Timer():
    predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k=k)

Elapsed: 0.22770094871520996


### Inference with Attention

In [34]:
import seaborn as sns
def show_attention(text, classifier, vectorizer, classes, max_length=dataset._max_seq_length, k=1):
    """Predict the rating of a review
    
    Args:
        text (str): the text of the description
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        classes (list of str): The name of the ouv classes
        k (int): show the largest k prediction, default to 1
    """
    ouv = preprocess_text(text)
    ouv_len = len(ouv.split())
    
    classifier.eval()
    state_word = classifier.init_hidden().to(args.device)
    vectorized_ouv = torch.tensor(vectorizer.vectorize(ouv, vector_length = max_length)[0])
    X = vectorized_ouv.view(1,-1).repeat(args.batch_size,1,1)
    result = classifier(vectorized_ouv.unsqueeze(0).repeat(args.batch_size,1),state_word, apply_softmax=True)[-1][0]
    attn = result.detach().numpy()
    
    tokens = ouv.split(' ')
    print_token = ['\033[0;39;44m'+tokens[i] if attn[i]>1.0/len(tokens) 
                   else '\033[0;39;107m'+tokens[i] if attn[i]>0.5/len(tokens)
                   else '\033[0;39;47m'+tokens[i] if attn[i]>0.1/len(tokens)
                   else '\033[0;39;49m'+tokens[i]
                   for i in range(len(tokens))]
    print(' '.join(print_token))
    
    return attn

In [35]:
args

Namespace(batch_size=256, bi=False, catch_keyboard_interrupt=True, cuda=False, device=device(type='cpu'), dropout_p=0, early_stopping_criteria=5, embedding_size=300, expand_filepaths_to_save_dir=True, freeze=True, frequency_cutoff=1, fuzzy=True, fuzzy_how='uni', fuzzy_lambda=0.2, glove_filepath='Data/glove/glove.6B.300d.txt', hidden_dim=128, k=3, l2=1e-05, learning_rate=0.001, model_state_file='model_storage/attn/model.pth', num_epochs=100, ouv_csv='Data/ouv_with_splits_full.csv', prior_csv='Data/Coappearance_matrix.csv', ratings_csv='Data/human_rates.csv', reload_from_files=False, save_dir='model_storage/attn/', seed=1337, use_glove=True, vectorizer_file='model_storage/attn/vectorizer.json')

In [36]:
k=3
show_attention('This is a very old building dating back to 13th century',classifier,vectorizer,classes,k=k)

this is a very old building dating back to th century


array([1.2716800e-03, 5.7590933e-04, 3.2928781e-04, 1.0889025e-03,
       3.8831106e-03, 9.5687523e-02, 6.2945044e-01, 5.8780748e-02,
       5.1390026e-02, 1.5178600e-02, 1.4236379e-01, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [37]:
k=3
show_attention('Saint Christopher, Saint Vincent Ferreri, Saint Sebastian (are depicted in this Altar)',classifier,vectorizer,classes,k=k)

saint christopher , saint vincent ferreri , saint sebastian are depicted in this altar 


array([0.04375633, 0.05424835, 0.01664908, 0.07758369, 0.02839746,
       0.02402568, 0.02167705, 0.08286478, 0.03810453, 0.01085976,
       0.01902153, 0.00453314, 0.00402649, 0.49807873, 0.07617339,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [38]:
show_attention('''The architecture of the Imperial Palace complexes, particularly in Shenyang, 
exhibits an important interchange of influences of traditional architecture and Chinese palace 
architecture particularly in the 17th and 18th centuries.''',classifier,vectorizer,classes,k=k)

the architecture of the imperial palace complexes , particularly in shenyang , exhibits an important interchange of influences of traditional architecture and chinese palace architecture particularly in the th and th centuries . 


array([2.1641505e-05, 1.5225343e-02, 3.6769002e-04, 1.2306761e-04,
       3.3024438e-02, 9.5179090e-03, 4.4805603e-03, 5.5463151e-03,
       3.5013244e-03, 1.3319124e-03, 3.4408979e-03, 2.7808642e-03,
       2.4274683e-03, 1.1310581e-03, 3.7345218e-03, 5.6605279e-03,
       1.3185071e-03, 4.6892059e-01, 4.1425726e-03, 3.5524447e-02,
       1.6723141e-01, 8.8114878e-03, 3.0805282e-02, 1.2297292e-02,
       1.3630727e-01, 1.7332496e-02, 4.0842174e-03, 1.3076827e-03,
       1.5618989e-03, 1.9770386e-03, 1.2900901e-03, 9.7581586e-03,
       2.4993871e-03, 2.5146264e-03, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [39]:
test = '''With the unusualness of an archaeological site which still breathes life, Venice bears testimony unto 
itself. This mistress of the seas is a link between the East and the West, between Islam and Christianity and 
lives on through thousands of monuments and vestiges of a time gone by.'''

In [40]:
show_attention(test,classifier,vectorizer,classes,k=k)

with the unusualness of an archaeological site which still breathes life , venice bears testimony unto itself . this mistress of the seas is a link between the east and the west , between islam and christianity and lives on through thousands of monuments and vestiges of a time gone by . 


array([3.5281535e-04, 5.1882460e-05, 4.6675003e-04, 9.4774259e-05,
       1.5358051e-04, 2.2895613e-01, 3.5995883e-03, 1.5663307e-03,
       4.2108442e-03, 4.1944478e-03, 1.3772404e-02, 5.7633081e-03,
       2.6178673e-02, 1.3602166e-02, 3.3615637e-01, 4.8800863e-02,
       1.3284966e-02, 1.5416360e-02, 1.8123721e-03, 1.0915331e-02,
       1.3309506e-03, 9.4688218e-04, 4.9420852e-02, 1.7434456e-03,
       2.9540571e-04, 1.0651515e-03, 4.0065896e-04, 2.0084185e-04,
       8.2686748e-03, 2.1177977e-03, 6.4169656e-04, 4.6805656e-03,
       7.7787614e-03, 2.2067758e-03, 8.7849200e-03, 3.3839729e-03,
       4.7267392e-02, 4.2197164e-03, 1.4599891e-02, 1.8816863e-03,
       7.7002859e-03, 6.1622863e-03, 1.0209952e-03, 8.3722593e-03,
       2.7325631e-03, 3.6731102e-02, 1.2392439e-03, 6.2849157e-04,
       4.4447095e-03, 8.7236138e-03, 1.4838366e-02, 9.1225700e-03,
       7.6975045e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [42]:
predict_rating(test,classifier,vectorizer,classes,k=3)

[('Criteria iii', 0.6642264127731323),
 ('Criteria vi', 0.19791539013385773),
 ('Criteria iv', 0.04785666987299919)]

In [43]:
predict_rating('''The architecture of the Imperial Palace complexes, particularly in Shenyang, 
exhibits an important interchange of influences of traditional architecture and Chinese palace 
architecture particularly in the 17th and 18th centuries.''',classifier,vectorizer,classes,k=k)

[('Criteria ii', 0.8414957523345947),
 ('Criteria iv', 0.10986685007810593),
 ('Criteria iii', 0.014939364045858383)]

In [48]:
sentence_f = '''the lagoon of venice also has one of the highest concentrations of masterpieces in the world'''
sentence_f1 = '''from torcellos cathedral to the church of santa maria della salute'''
sentence_g = '''the years of the republics extraordinary golden age are represented by monuments of incomparable beauty'''
sentence_h = '''san marco , palazzo ducale , san zanipolo , scuola di san marco , frari and scuola di san rocco , san giorgio maggiore , etc'''

In [49]:
show_attention(sentence_f,classifier,vectorizer,classes)

the lagoon of venice also has one of the highest concentrations of masterpieces in the world


array([8.9620677e-05, 1.0345962e-02, 2.8579152e-04, 7.6073254e-03,
       6.3028967e-04, 5.3201697e-04, 9.5164462e-05, 1.2912272e-04,
       9.8319950e-05, 9.8977226e-04, 1.9344218e-03, 1.4400504e-04,
       9.6901965e-01, 4.6469267e-03, 1.9053022e-04, 3.2611040e-03,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [51]:
show_attention(sentence_f1,classifier,vectorizer,classes)

from torcellos cathedral to the church of santa maria della salute


array([0.01298109, 0.01378309, 0.0169484 , 0.01023767, 0.00248664,
       0.25505283, 0.01498654, 0.02350277, 0.38028172, 0.19383384,
       0.07590542, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [52]:
show_attention(sentence_g,classifier,vectorizer,classes)

the years of the republics extraordinary golden age are represented by monuments of incomparable beauty


array([2.51027581e-04, 5.72370132e-04, 1.19964141e-04, 6.98789736e-05,
       9.44014173e-04, 7.49043375e-03, 1.66301891e-01, 2.30476528e-01,
       8.18199571e-03, 1.64250173e-02, 3.22422758e-02, 3.69247682e-02,
       2.32940912e-03, 1.03519686e-01, 3.94150734e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [63]:
test = '''The designated property area includes all elements embodying the values in the creativity, influence, 
historic evidence, and architectural exemplar, with the historical scale, architectural types, and other 
components, as well as the techniques and artistic achievements of Chinese palace buildings after the 15th 
century, particularly in the 17th to 18th century, well preserved'''

In [53]:
predict_rating(sentence_f,classifier,vectorizer,classes,k=3)

[('Criteria i', 0.733273983001709),
 ('Criteria vii', 0.10703416168689728),
 ('Criteria viii', 0.03875169903039932)]

In [54]:
predict_rating(sentence_f1,classifier,vectorizer,classes,k=3)

[('Criteria vi', 0.3464215397834778),
 ('Criteria iv', 0.2133128046989441),
 ('Criteria ii', 0.1942373812198639)]

In [55]:
predict_rating(sentence_g,classifier,vectorizer,classes,k=3)

[('Criteria i', 0.41988039016723633),
 ('Criteria iii', 0.1634589433670044),
 ('Criteria vi', 0.15812163054943085)]

In [57]:
predict_rating(sentence_f + sentence_f1 + sentence_g,classifier,vectorizer,classes,k=3)

[('Criteria i', 0.7399744987487793),
 ('Criteria vii', 0.07228098809719086),
 ('Criteria iv', 0.04865434020757675)]

### Producing Highlighted Ratings for Human Study

In [51]:
rates = pd.read_csv(args.ratings_csv, sep ='\t')

In [50]:
k=3
for i in range(len(rates)):
    show_attention(rates['data'].iloc[i],classifier,vectorizer,classes,k=k)
    print()

with the unusualness of an archaeological site which still breathes life , venice bears testimony unto itself

venice symbolizes the peoples victorious struggle against the elements as they managed to master a hostile nature

it was from venice that marco polo num num set out in search of china , annam , tonkin , sumatra , india and persia

the queen of the seas , heroically perched on her tiny islands , extended her horizon well beyond the lagoon , the adriatic and the mediterranean

this mistress of the seas is a link between the east and the west , between islam and christianity and lives on through thousands of monuments and vestiges of a time gone by . 

the city is also directly and tangibly associated with the history of humankind

in the mediterranean area , the lagoon of venice represents an outstanding example of a semi lacustral habitat which has become vulnerable as a result of irreversible natural and climate changes

venice possesses an incomparable series of architectura


venice and its lagoon form an inseparable whole of which the city of venice is the pulsating historic heart and a unique artistic achievement

human interventions show high technical and creative skills in the realization of the hydraulic and architectural works in the lagoon area

over the centuries , during the entire period of the expansion of venice , when it was obliged to defend its trading markets against the commercial undertakings of the arabs , the genoese and the ottoman turks , venice never ceased to consolidate its position in the lagoon . 

the unique cultural heritage accumulated in the lagoon over the centuries is attested by the discovery of important archaeological settlements in the altino area and other sites on the mainland , which were important communication and trade hubs . 

venice and its lagoon landscape is the result of a dynamic process which illustrates the interaction between people and the ecosystem of their natural environment over time

when a group o


saint christopher , saint vincent ferreri , saint sebastian ss

structural model of the basilica showing a cross section of the central part of the church with the domes of the ascension , the chancel and the south transept

sculpted by pietro da sal in , the statue presents a crouching , naked hunchback supporting a small flight of steps

if the buildings would be straight , painted well maintained , i think venice without this patina , the gondolas gondolieri , channels , little bridges the at all time busy canal grande all these main ingredients altogether express the romantic charm of venice

mark s basilic until , after restoration they were then moved to the museum in for reasons of conservation and replaced by reproductions it was made by lost wax casting using the so called indirect method , in an alloy with a very high percentage of copper , a requirement for the procedure of mercury gilding

the venetian arsenal italian arsenale di venezia is a complex of former shipyards an

on the altar , between a rich and elegant cornice , probably sculpted by jacopo from faenza , there is a beautiful polyptych signed by bartolom eo vivarini 

 as satire was forbidden by law in veneto , people wrote poems and stories against the nobility which was the only one to hold power and put them up on the statue signing them with its name

 it is the tallest structure in venice and is colloquially termed el par n de casa the master of the house 

beeing on my way to the hotel in the early morning , i took a picture from the basilica di santa maria della salute immediately after arriving in venice

the enclosed bridge is made of white limestone , has windows with stone bars , passes over the rio di palazzo , and connects the new prison prigioni nuove to the interrogation rooms in the doge s palace



## Interpretability

### Experimenting with the effect of Embedding fine-tuning

In [66]:
ouv_df = pd.read_csv(args.ouv_csv)
word_counts = Counter()
for data in ouv_df.data:
    for word in data.split(' '):
        if word not in string.punctuation:
            word_counts[word] += 1

In [67]:
def ngrams_iterator(token_list, ngrams):
    def _get_ngrams(n):
        return zip(*[token_list[i:] for i in range(n)])
    for x in token_list:
        yield x
    for n in range(2, ngrams+1):
        for x in _get_ngrams(n):
            yield ' '.join(x)

In [68]:
ouv_df = pd.read_csv(args.ouv_csv)
word_counts = Counter()
for data in ouv_df.data:
    token_list = data.split(' ')
    for word in ngrams_iterator(token_list, 5):
        temp = 0
        for element in word:
            if element in string.punctuation:
                temp = 1
                break
        if temp==0:
            word_counts[word] += 1

In [69]:
vocab = [word for word, count in word_counts.items() if count>15 and count<600]
len(vocab)

2353

In [70]:
def infer_tokens_importance(vocab, classifier, vectorizer, classes, k=50):
    """Predict the rating of a review
    
    Args:
        vocab (list of str): the whole vocabulary
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        classes (list of str): The name of the ouv classes
        k (int): show the largest k prediction, default to 1
    """
    vectorized_token = []    
    for token in vocab:
        vectorized_token.append(torch.tensor(vectorizer.vectorize(token, vector_length=dataset._max_seq_length)[0]))
    
    result=torch.zeros((int(len(vocab)/args.batch_size)+1)*args.batch_size,len(classes))
    #print(result.shape)
    for i in range(int(len(vocab)/args.batch_size)+1):
        state_word = classifier.init_hidden()
        X = torch.stack(vectorized_token[i*args.batch_size:(i+1)*args.batch_size])
        X = torch.cat([X,torch.zeros(args.batch_size-X.shape[0], dataset._max_seq_length).long()])
        #print(X.shape)
        classifier.eval()
        res,state_word,_ = classifier(X,state_word, apply_softmax=True)
        #print(res.shape)
        result[i*args.batch_size:(i+1)*args.batch_size]=res
        #print(result.shape)
    result = result[:len(vocab)]
    
    vocab_id = result[1:].topk(k, dim=0)[1]
    vocab_weight = result[1:].topk(k, dim=0)[0]
    return vocab_id, vocab_weight

In [71]:
all_k = infer_tokens_importance(vocab, classifier, vectorizer, classes, k=50)

In [73]:
def make_top_k_DataFrame(vocab, classifier, vectorizer, classes, k=10):
    
    vocab_id = infer_tokens_importance(vocab, classifier, vectorizer, classes, k)[0]
    df = pd.DataFrame(columns = classes)
    for i in range(len(classes)):
        
        indices = vocab_id[:,i].tolist()
        words = pd.Series([vocab[j] for j in indices])
        df[classes[i]] = words
    return df

In [74]:
make_top_k_DataFrame(vocab, classifier, vectorizer, classes, k=20)

,Criteria i,Criteria ii,Criteria iii,Criteria iv,Criteria v,Criteria vi,Criteria vii,Criteria viii,Criteria ix,Criteria x,Others
0,of human creative,more than,of early,example of a type,a period,political,the great,impressive,is one,include the,royal
1,a unique and,manner,the archaeological,dominant,a cultural,indigenous,exceptional natural,this region,and biological,bird species,in the area
2,the exceptional,an exceptional,archaeological site of,styles,the cultural landscape,is still,of exceptional,the architecture of,zone,exhibit,erosion
3,a masterpiece of,interchange of,culture of,colonization,design of,holy,geological,of monuments,geological processes,bird,include
4,unique artistic achievement,on the development of,culture of the,are outstanding examples of,bronze age,testify to,man,succession,processes,critically endangered,immense
5,masterpiece of,architecture in the,exceptional testimony to the,example of an architectural,ad,impact,trees,features,back,greatest,rome
6,a unique artistic,development of architecture,over a period,outstanding example of a,is an,islam,glaciers,changes,surrounded,plant and,wild
7,decoration,profound influence,the modern,outstanding example of the,of a type of,the most complete,deep,basin,the development of the,also contains,the eastern
8,is a masterpiece,profound influence on,bears exceptional testimony,outstanding examples of,deciduous,example of a traditional,altitude,record,worldwide,critically,north america
9,artistic achievement,exerted,bears exceptional testimony to,outstanding example of an,the landscape,documented,metres,i,series,the islands,time


In [75]:
make_top_k_DataFrame(vocab, classifier, vectorizer, classes, k=50).to_csv(args.save_dir+'top_words.csv')

## Confusion Matrix

In [160]:
dataset.set_split('test')
set_seed_everywhere(args.seed, args.cuda)
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
conf_mat_test = np.zeros((len(classes)-1,len(classes)-1))

In [161]:
for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # compute the output
    classifier.eval()
    state_word = classifier.init_hidden()
    y_pred,state_word,_ = classifier(X,state_word)
    
    conf_mat_test = np.add(conf_mat_test,confusion_matrix(Y.argmax(axis=1), y_pred.argmax(axis=1)))

In [146]:
conf_mat_test

array([[20.,  3.,  2.,  6.,  1.,  3.,  1.,  0.,  0.,  0.],
       [ 1., 46.,  4., 13.,  4.,  4.,  1.,  1.,  0.,  0.],
       [ 1.,  5., 40., 11.,  3.,  2.,  1.,  0.,  0.,  0.],
       [ 3., 13., 11., 45.,  6.,  0.,  2.,  1.,  0.,  0.],
       [ 0.,  0.,  3., 13., 12.,  0.,  2.,  0.,  2.,  0.],
       [ 0.,  4., 11.,  4.,  1., 22.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  2.,  1., 30.,  4.,  1.,  4.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  5., 18.,  5.,  2.],
       [ 0.,  0.,  2.,  0.,  1.,  0.,  3.,  1., 28., 12.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  7., 54.]])

In [162]:
dataset.set_split('val')
set_seed_everywhere(args.seed, args.cuda)
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
conf_mat_val = np.zeros((len(classes)-1,len(classes)-1))

In [163]:
for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # compute the output
    state_word = classifier.init_hidden()
    y_pred,state_word,_ = classifier(X,state_word)
    
    conf_mat_val = np.add(conf_mat_val,confusion_matrix(Y.argmax(axis=1), y_pred.argmax(axis=1)))

In [149]:
conf_mat_val

array([[14.,  5.,  3.,  9.,  0.,  4.,  0.,  0.,  0.,  0.],
       [ 3., 39.,  6., 12.,  1.,  5.,  0.,  0.,  0.,  0.],
       [ 3.,  5., 51., 10.,  3.,  7.,  0.,  0.,  0.,  0.],
       [ 1., 14.,  9., 45.,  7.,  5.,  0.,  1.,  0.,  0.],
       [ 1.,  1.,  2.,  7.,  9.,  1.,  1.,  1.,  2.,  0.],
       [ 0.,  6.,  5.,  1.,  0., 28.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0., 31.,  1.,  2.,  4.],
       [ 0.,  0.,  1.,  1.,  0.,  1.,  6., 26.,  3.,  2.],
       [ 0.,  0.,  1.,  0.,  1.,  0.,  2.,  2., 22., 12.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  3.,  0.,  5., 57.]])

In [164]:
dataset.set_split('train')
set_seed_everywhere(args.seed, args.cuda)
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
conf_mat_train = np.zeros((len(classes)-1,len(classes)-1))

In [165]:
for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # compute the output
    state_word = classifier.init_hidden()
    y_pred,state_word,_ = classifier(X,state_word)
    
    conf_mat_train = np.add(conf_mat_train,confusion_matrix(Y.argmax(axis=1), y_pred.argmax(axis=1),labels=range(len(classes)-1)))

In [153]:
conf_mat_train

array([[196.,  26.,  10.,  76.,   2.,   8.,   2.,   0.,   0.,   0.],
       [ 20., 402.,  35., 110.,   9.,  28.,   0.,   0.,   0.,   0.],
       [ 16.,  51., 388.,  95.,  24.,  49.,   2.,   3.,   2.,   3.],
       [ 25.,  94.,  53., 529.,  19.,  26.,   1.,   2.,   1.,   0.],
       [  2.,   8.,  24.,  41., 117.,   1.,   3.,   2.,   4.,   1.],
       [  6.,  32.,  49.,  12.,   4., 209.,   2.,   0.,   1.,   0.],
       [  1.,   0.,   3.,   0.,   4.,   1., 277.,  25.,  23.,  34.],
       [  0.,   0.,   0.,   0.,   1.,   1.,  30., 203.,  15.,   3.],
       [  0.,   0.,   1.,   0.,   0.,   0.,  19.,   9., 242.,  84.],
       [  0.,   0.,   1.,   0.,   0.,   1.,  13.,   2.,  33., 501.]])

In [154]:
pd.concat([pd.DataFrame(conf_mat_test),pd.DataFrame(conf_mat_val),pd.DataFrame(conf_mat_train)],axis=1).to_csv(args.save_dir+'confusion_matrix.csv')

In [166]:
pd.concat([pd.DataFrame(conf_mat_test),pd.DataFrame(conf_mat_val),pd.DataFrame(conf_mat_train)],axis=1).to_csv(args.save_dir+'baseline_confusion_matrix.csv')

In [167]:
def per_class_metrics(confusion_matrix, classes):
    '''
    Compute the per class precision, recall, and F1 for all the classes
    
    Args:
    confusion_matrix (np.ndarry) with shape of (n_classes,n_classes): a confusion matrix of interest
    classes (list of str) with shape (n_classes,): The names of classes
    
    Returns:
    metrics_dict (dictionary): a dictionary that records the per class metrics
    '''
    num_class = confusion_matrix.shape[0]
    metrics_dict = {}
    for i in range(num_class):
        key = classes[i]
        temp_dict = {}
        row = confusion_matrix[i,:]
        col = confusion_matrix[:,i]
        val = confusion_matrix[i,i]
        precision = val/row.sum()
        recall = val/col.sum()
        F1 = 2*(precision*recall)/(precision+recall)
        temp_dict['precision'] = precision
        temp_dict['recall'] = recall
        temp_dict['F1'] = F1
        metrics_dict[key] = temp_dict
    
    return metrics_dict

In [168]:
metrics_dict = {}
metrics_dict['test'] = per_class_metrics(conf_mat_test, classes[:-1])
metrics_dict['val'] = per_class_metrics(conf_mat_val, classes[:-1])
metrics_dict['train'] = per_class_metrics(conf_mat_train, classes[:-1])

In [169]:
metrics_df = pd.DataFrame.from_dict({(i,j): metrics_dict[i][j] 
                           for i in metrics_dict.keys() 
                           for j in metrics_dict[i].keys()},
                       orient='index')

In [158]:
metrics_df.to_csv(args.save_dir+'per_class_metrics.csv')

In [170]:
metrics_df.to_csv(args.save_dir+'baseline_per_class_metrics.csv')

## Try on totally Unseen Data

In [105]:
#args.ouv_csv='Data/ouv_with_splits_full.csv',
new_ouv_csv='Data/sd_full.csv'
#args.reload_from_files=True

In [107]:
def compute_jac_k_accuracy(y_pred, y_target, k=3, multilabel=False):
    
    y_pred_indices = y_pred.topk(k, dim=1)[1]
    y_target_indices = y_target.topk(k, dim=1)[1]
        
    n_correct = torch.tensor([torch.tensor([y_pred_indices[j][i] in y_target_indices[j] for i in range(k)]).sum()>0 
                              for j in range(len(y_pred))]).sum().item()
    return n_correct / len(y_pred_indices) * 100

def compute_jac_1_accuracy(y_pred, y_target, k=3, multilabel=False):
    
    y_pred_indices = y_pred.topk(1, dim=1)[1]
    y_target_indices = y_target.topk(k, dim=1)[1]
        
    n_correct = torch.tensor([torch.tensor([y_pred_indices[j] in y_target_indices[j] for i in range(k)]).sum()>0 
                              for j in range(len(y_pred))]).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [173]:
with Timer():
    loss_func = cross_entropy
    set_seed_everywhere(args.seed, args.cuda)
    train_state = make_train_state(args)
    dataset = OuvDataset.load_dataset_and_load_vectorizer(new_ouv_csv, args.vectorizer_file)

    dataset.set_split('val')
    verbose=False
    try:
        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_1_acc = 0.0
        running_k_acc = 0.0
        running_k_jac = 0.0
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # step 2. get the data compute fuzzy labels
            state_word = classifier.init_hidden().to(args.device)
            X = batch_dict['x_data']

            y_target = batch_dict['y_target']
            y_fuzzy = batch_dict['y_fuzzy']

            Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                                    how=args.fuzzy_how, lbd = args.fuzzy_lambda)

            # step 3. compute the output
            with torch.no_grad():
                y_pred,state_word,_ = classifier(X, state_word)

            # step 4. compute the loss
            loss = loss_func(y_pred, Y)
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # -----------------------------------------
            # compute the accuracies
            acc_1_t = compute_jac_1_accuracy(y_pred, y_target)
            acc_k_t = compute_jac_k_accuracy(y_pred, y_target, args.k)
            jac_k_t = compute_jaccard_index(y_pred, y_target, len(classes))

            running_1_acc += (acc_1_t - running_1_acc) / (batch_index + 1)
            running_k_acc += (acc_k_t - running_k_acc) / (batch_index + 1)
            running_k_jac += (jac_k_t - running_k_jac) / (batch_index + 1)

            # update bar
            if verbose:
                val_bar.set_postfix(loss=running_loss, 
                                acc_1=running_1_acc,
                                acc_k=running_k_acc,
                                jac_k=running_k_jac,
                                epoch=epoch_index)
                val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_1_acc'].append(running_1_acc)
        train_state['val_k_acc'].append(running_k_acc)
        train_state['val_k_jac'].append(running_k_jac)

        if verbose:
            train_bar.n = 0
            val_bar.n = 0
            epoch_bar.update()

    except KeyboardInterrupt:
        print("Exiting loop")
        pass

Elapsed: 5.034240961074829


In [114]:
train_state

{'stop_early': False,
 'early_stopping_step': 0,
 'early_stopping_best_k_acc_val': 0,
 'learning_rate': 0.001,
 'epoch_index': 0,
 'train_loss': [],
 'train_1_acc': [],
 'train_k_acc': [],
 'train_k_jac': [],
 'val_loss': [2.016925685455641],
 'val_1_acc': [66.35416666666669],
 'val_k_acc': [94.0625],
 'val_k_jac': [0.4140845318635305],
 'test_loss': -1,
 'test_1_acc': -1,
 'test_k_acc': -1,
 'test_k_jac': -1,
 'model_filename': 'model_storage/attn/model.pth'}

In [174]:
train_state

{'stop_early': False,
 'early_stopping_step': 0,
 'early_stopping_best_k_acc_val': 0,
 'learning_rate': 0.001,
 'epoch_index': 0,
 'train_loss': [],
 'train_1_acc': [],
 'train_k_acc': [],
 'train_k_jac': [],
 'val_loss': [2.4428400123202985],
 'val_1_acc': [64.27083333333333],
 'val_k_acc': [92.70833333333334],
 'val_k_jac': [0.3412336905797323],
 'test_loss': -1,
 'test_1_acc': -1,
 'test_k_acc': -1,
 'test_k_jac': -1,
 'model_filename': 'model_storage/attn/model.pth'}

## END